## 4. Случайный лес. Практика

Пожалуй, каждый из нас практически ежедневно смотрит прогноз погоды: будет сегодня тепло или холодно, брать ли зонт? Мы расстраиваемся, если внезапно идёт дождь, а мы оказались к этому не готовы. Иногда можно понять, что будет дождь, просто взглянув на небо, но часто такие предположения оказываются неверными. Надёжнее всего пользоваться прогнозами, которые публикуют специалисты. А задумывались ли вы, как формируются эти прогнозы?

Да-да, здесь тоже не обошлось без машинного обучения. В этом юните мы с вами немного коснёмся метеорологии, чтобы предсказать, будет ли дождь в Австралии. Попутно вы узнаете, какие факторы влияют на вероятность дождя — возможно, вы научитесь предсказывать его точнее, чем метеорологические службы, и больше никогда не окажетесь в нужный момент без зонта.

Данные содержат 23 признака и 145 460 наблюдений. Из этих 23 признаков шесть — категориальные, в одном записана дата, а остальные являются непрерывными числовыми данными.

Примеры числовых признаков: температура, скорость ветра, влажность, облачность, атмосферное давление в разное время суток, количество осадков, испарение, количество часов с солнечной погодой.
Примеры категориальных признаков: местоположение, направление ветра в разное время суток, наличие дождя сегодня или завтра.
Целевой переменной является столбец RainTomorrow. Значение этой переменной мы и будем пытаться предсказать.

# Подробная расшифровка всех признаков
- Date — дата, в которую зафиксировано наблюдение
- Location — местонахождение метеорологической станции
- MinTemp — минимальная температура (℃)
- MaxTemp — максимальная температура (℃)
- Rainfall — количество осадков (дождь) за сутки (мм)
- Evaporation — количество испарений до 9 утра (мм)
- Sunshine — количество часов в сутках, когда светило солнце
- WindGustDir — направление самого сильного порыва ветра за последние 24 часа
- WindGustSpeed — скорость самого сильного порыва ветра за последние 24 часа
- WindDir9am — направление ветра в 9 утра
- WindDir3pm — направление ветра в 3 часа дня
- WindSpeed9am — скорость ветра в 9 часов утра
- WindSpeed3pm — скорость ветра в 3 часа дня
- Humidity9am — влажность в 9 утра
- Humidity3pm — влажность в 3 часа дня
- Pressure9am — атмосферное давление в 9 утра
- Pressure3pm — атмосферное давление в 3 часа дня
- Cloud9am — часть неба, закрытая облаками, в 9 утра
- Cloud3pm — часть неба, закрытая облаками, в 3 часа дня
- Temp9am — температура в 9 утра
- Temp3pm — температура в 3 часа дня
- RainToday — наличие дождя в этот день
- RainTomorrow — наличие дождя на следующий день.

In [74]:
import numpy as np
import pandas as pd
from sklearn import linear_model #линейные моделиё
from sklearn.linear_model import LogisticRegression
from sklearn.linear_model import LinearRegression
from sklearn import tree #деревья решений
from sklearn import ensemble #ансамбли
from sklearn import metrics #метрики
from sklearn import preprocessing #предобработка
from sklearn.model_selection import train_test_split #сплитование выборки
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import f1_score
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import roc_auc_score
from sklearn.tree import DecisionTreeClassifier
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import BaggingClassifier
from sklearn.ensemble import RandomForestClassifier


%matplotlib inline

In [56]:
df = pd.read_csv('weatherAUS.csv')

ЗАДАНИЕ 4.1

In [57]:
df.isnull().sum().sum()

343248

ЗАДАНИЕ 4.2

In [58]:
print(round((df.isna().sum() / len(df))*100, 1))
df.drop(['Evaporation','Sunshine','Cloud3pm'], axis = 1, inplace = True)

Date              0.0
Location          0.0
MinTemp           1.0
MaxTemp           0.9
Rainfall          2.2
Evaporation      43.2
Sunshine         48.0
WindGustDir       7.1
WindGustSpeed     7.1
WindDir9am        7.3
WindDir3pm        2.9
WindSpeed9am      1.2
WindSpeed3pm      2.1
Humidity9am       1.8
Humidity3pm       3.1
Pressure9am      10.4
Pressure3pm      10.3
Cloud9am         38.4
Cloud3pm         40.8
Temp9am           1.2
Temp3pm           2.5
RainToday         2.2
RainTomorrow      2.2
dtype: float64


ЗАДАНИЕ 4.3

In [59]:
df.RainToday = df.RainToday.map({'No': 0, 'Yes': 1})
df.RainTomorrow = df.RainTomorrow.map({'No': 0, 'Yes': 1})
print(round(df['RainToday'].mean(), 2))

0.22


ЗАДАНИЕ 4.4

In [60]:
df.Date = pd.to_datetime(df.Date)
df['Month'] = df.Date.dt.month
df.drop('Date', axis = 1, inplace = True)
df_season = df.groupby('Month').mean()
df_season[['RainToday']]

C:\Users\Psa1kl\AppData\Local\Temp\ipykernel_17116\1749877501.py:4: FutureWarning: The default value of numeric_only in DataFrameGroupBy.mean is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  df_season = df.groupby('Month').mean()


,RainToday
Month,
1,0.189484
2,0.206746
3,0.217135
4,0.216845
5,0.222163
6,0.263638
7,0.270736
8,0.253167
9,0.229135



ЗАДАНИЕ 4.5

In [63]:
categoricals = ['Month', 'Location', 'WindGustDir', 'WindDir9am', 'WindDir3pm']
df_dummies = pd.get_dummies(df, columns=categoricals)
df_dummies.shape

(145460, 124)

ЗАДАНИЕ 4.6

In [64]:
df_dummies.dropna(inplace=True)
X = df_dummies.drop('RainTomorrow', axis = 1)
Y = df_dummies['RainTomorrow']  
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.3, random_state = 31)
Y_test.mean()

0.22770253002811142

ЗАДАНИЕ 4.7

In [65]:
def gbs(data, n):     
    inds = np.random.randint(0, len(data), (n, len(data))) #определяем индексы случайным образом
    numbers = data[inds] #выбираем значения по индексам
    return numbers
target = X_train['MinTemp'].values #выбираем целевую переменную
np.random.seed(31) #задаём параметр генератора случайных чисел
mean_values = [np.mean(x) for x in gbs(target, 1000)] #получаем все средние значения
np.std(mean_values) #находим для них стандартное отклонение

0.02879072820657669

ЗАДАНИЕ 4.8

In [69]:
clf = LogisticRegression()
clf.fit(X_train, Y_train)
preds_train = clf.predict(X_train)
preds_test = clf.predict(X_test)
roc_auc_score(Y_test, preds_test)

c:\Users\Psa1kl\AppData\Local\Programs\Python\Python39\lib\site-packages\sklearn\linear_model\_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


0.7263153819242503

ЗАДАНИЕ 4.9

In [72]:
params = {
    'max_leaf_nodes': list(range(2, 10)), 
    'min_samples_split': [2, 3, 4], 
    'max_depth': [5,7,9,11]
}

grid_search_cv = GridSearchCV(DecisionTreeClassifier(random_state=42), params, verbose=3, cv=3)
grid_search_cv.fit(X_train, Y_train)
print(grid_search_cv.best_params_)

clf = DecisionTreeClassifier(max_depth = 5, max_leaf_nodes = 9, min_samples_split = 2, random_state=42)
clf.fit(X_train, Y_train)
preds_train = clf.predict(X_train)
preds_test = clf.predict(X_test)
print(round(roc_auc_score(Y_test, preds_test), 2))

Fitting 3 folds for each of 96 candidates, totalling 288 fits
[CV 1/3] END max_depth=5, max_leaf_nodes=2, min_samples_split=2;, score=0.817 total time=   0.1s
[CV 2/3] END max_depth=5, max_leaf_nodes=2, min_samples_split=2;, score=0.820 total time=   0.0s
[CV 3/3] END max_depth=5, max_leaf_nodes=2, min_samples_split=2;, score=0.825 total time=   0.1s
[CV 1/3] END max_depth=5, max_leaf_nodes=2, min_samples_split=3;, score=0.817 total time=   0.0s
[CV 2/3] END max_depth=5, max_leaf_nodes=2, min_samples_split=3;, score=0.820 total time=   0.0s
[CV 3/3] END max_depth=5, max_leaf_nodes=2, min_samples_split=3;, score=0.825 total time=   0.0s
[CV 1/3] END max_depth=5, max_leaf_nodes=2, min_samples_split=4;, score=0.817 total time=   0.0s
[CV 2/3] END max_depth=5, max_leaf_nodes=2, min_samples_split=4;, score=0.820 total time=   0.0s
[CV 3/3] END max_depth=5, max_leaf_nodes=2, min_samples_split=4;, score=0.825 total time=   0.0s
[CV 1/3] END max_depth=5, max_leaf_nodes=3, min_samples_split=2;,

ЗАДАНИЕ 4.10

In [75]:
clf = RandomForestClassifier(n_estimators = 100, random_state=31)
clf.fit(X_train, Y_train)
preds_train = clf.predict(X_train)
preds_test = clf.predict(X_test)
print(round(roc_auc_score(Y_test, preds_test), 2))

0.73


ЗАДАНИЕ 4.11

In [76]:
params = {
    'max_features': [4, 5, 6, 7], 
    'min_samples_leaf': [3, 5, 7, 9, 11], 
    'max_depth': [5, 10, 15]
}
grid_search_cv = GridSearchCV(RandomForestClassifier(random_state=31), params, verbose=3, cv=3)
grid_search_cv.fit(X_train, Y_train)
print(grid_search_cv.best_params_)
clf = RandomForestClassifier(max_depth=15, max_features=7, min_samples_leaf=3, random_state=31)
clf.fit(X_train, Y_train)
preds_train = clf.predict(X_train)
preds_test = clf.predict(X_test)
print(round(roc_auc_score(Y_test, preds_test), 2))

Fitting 3 folds for each of 60 candidates, totalling 180 fits
[CV 1/3] END max_depth=5, max_features=4, min_samples_leaf=3;, score=0.776 total time=   1.1s
[CV 2/3] END max_depth=5, max_features=4, min_samples_leaf=3;, score=0.775 total time=   0.9s
[CV 3/3] END max_depth=5, max_features=4, min_samples_leaf=3;, score=0.775 total time=   0.9s
[CV 1/3] END max_depth=5, max_features=4, min_samples_leaf=5;, score=0.776 total time=   0.9s
[CV 2/3] END max_depth=5, max_features=4, min_samples_leaf=5;, score=0.775 total time=   1.0s
[CV 3/3] END max_depth=5, max_features=4, min_samples_leaf=5;, score=0.775 total time=   0.9s
[CV 1/3] END max_depth=5, max_features=4, min_samples_leaf=7;, score=0.776 total time=   1.0s
[CV 2/3] END max_depth=5, max_features=4, min_samples_leaf=7;, score=0.775 total time=   1.1s
[CV 3/3] END max_depth=5, max_features=4, min_samples_leaf=7;, score=0.775 total time=   1.2s
[CV 1/3] END max_depth=5, max_features=4, min_samples_leaf=9;, score=0.776 total time=   0.9

ЗАДАНИЕ 4.12

In [77]:
feature_names = [x for x in df_dummies if x != 'RainTomorrow']
pd.DataFrame({'feat': feature_names,
              'coef': clf.feature_importances_}).sort_values(by='coef', ascending=False)

,feat,coef
7,Humidity3pm,0.250783
2,Rainfall,0.079757
6,Humidity9am,0.070403
10,Cloud9am,0.067092
9,Pressure3pm,0.065272
...,...,...
50,Location_Newcastle,0.000000
62,Location_SalmonGums,0.000000
51,Location_Nhil,0.000000
52,Location_NorahHead,0.000000
